<a href="https://colab.research.google.com/github/gantir/eva4/blob/develop/ds_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install dload

In [0]:
import os
import dload
import csv
import shutil
import time
from PIL import Image,ImageOps
from math import ceil
import numpy as np
import cv2
import random
import copy
import logging
import zipfile

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')
data_root = '/content/gdrive/My Drive/Colab Notebooks/EVA4/s14-15/data'

In [0]:
!mkdir logs

In [0]:
logging.basicConfig(filename='logs/debug.log',
  filemode='a',
  format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
  datefmt='%H:%M:%S',
  level=logging.DEBUG
)

logger = logging.getLogger(__name__)

def download_images(img_url_file: str, img_dest: str):
  src_filepath = os.path.join(os.getcwd(), img_url_file)
  dest_filepath = os.path.join(os.getcwd(), "img_src_local_map.csv")
  with open(src_filepath) as s, open(dest_filepath,'w') as d:
    dest_file = csv.writer(d,lineterminator=os.linesep, quoting=csv.QUOTE_ALL)
    urls = sorted(set(s.readlines()))
    for i, img_url in enumerate(urls):
      img_url = img_url.strip()
      dest_image_name = "img{:04d}.jpg".format(i+1)
      dest_image_path = os.path.join(os.getcwd(), img_dest, dest_image_name)
      try:
        print("Downloading as {} image {}".format(dest_image_name, img_url))
        dest_file.writerow([dest_image_name,img_url])
        dload.save(img_url, dest_image_path)
        if "pixabay" in img_url:
          # This delay is needed as otherwise the pixabay was throwing 403 error
          time.sleep(5)
      except Exception as e:
        print("Error: Error downloading as {} image {}".format(dest_image_name, img_url))
        pass

def alter_bg_image(src_img_dir: str, src_img_filename: str, dest_img_folder: str, image_size=(224,224), convert_gray_scale=False ):
  image_path = os.path.join(src_img_dir,src_img_filename)
  org_img = Image.open(image_path)
  if convert_gray_scale:
    org_img = ImageOps.grayscale(org_img)
  altered_image = ImageOps.fit(org_img,image_size, method=Image.ANTIALIAS)
  # altered_image.show()
  altered_image.save(os.path.join(dest_img_folder,src_img_filename))

def alter_fg_image(src_img_dir, src_img_filename, dest_img_folder, dest_img_folder_gray, max_size=200):
  image_path = os.path.join(src_img_dir,src_img_filename)
  try:
    # https://pillow.readthedocs.io/en/latest/handbook/concepts.html#modes
    org_img = Image.open(image_path)
    org_img_gray = org_img.convert('LA')

    width,height = org_img.size
    ratio = width/height
    new_width = max_size
    new_height = ceil(max_size/ratio)
    if width < height:
      new_width = ceil(max_size*ratio)
      new_height = max_size
    # print(ratio,src_img_filename,[width,height],[new_width,new_height])
    altered_image = ImageOps.fit(org_img,(new_width,new_height), method=Image.ANTIALIAS)
    altered_image.save(os.path.join(dest_img_folder,src_img_filename))

    altered_image_gray = ImageOps.fit(org_img_gray,(new_width,new_height), method=Image.ANTIALIAS)
    altered_image_gray.save(os.path.join(dest_img_folder_gray,src_img_filename))
  except Exception as e:
    print(e)
    pass

def rename_bg_files(src_img_dir, dest_img_dir):
  img_files = sorted(os.listdir(src_img_dir))
  for i, img in enumerate(img_files):
    dest_image_name = "img{:04d}.jpg".format(i+1)
    src_img_path = os.path.join(src_img_dir, img)
    dest_image_path = os.path.join(dest_img_dir, dest_image_name)
    os.rename(src_img_path, dest_image_path)

def rename_fg_files(src_dir, dest_img_dir):
  # classes = ['bird','boat','car','cat','cow','dog','person']
  classes = ['person']
  img_files = []
  for cl in classes:
    for img in os.listdir(os.path.join(src_dir,cl)):
      if 'png' in img:
        img_files.append((cl,os.path.join(src_dir,cl,img)))

  class_img = []
  for i, details in enumerate(img_files):
    dest_image_name = "img{:04d}.png".format(i+1)
    dest_image_path = os.path.join(dest_img_dir, dest_image_name)
    shutil.copy2(details[1],dest_image_path)
    class_img.append((details[0],dest_image_name))

  with open('/Users/projects/Downloads/s14-15/fg_img_class_map.csv','w') as f:
    fg_map_file = csv.writer(f,lineterminator=os.linesep, quoting=csv.QUOTE_ALL)
    fg_map_file.writerows(class_img)

def generate_mask(src_img_dir, dest_img_dir):
  img_files = sorted(os.listdir(src_img_dir))
  for img in img_files:
    # https://stackoverflow.com/questions/28430904/set-numpy-array-elements-to-zero-if-they-are-above-a-specific-threshold
    # https://codereview.stackexchange.com/questions/184044/processing-an-image-to-extract-green-screen-mask
    image = cv2.imread(os.path.join(src_img_dir,img))
    white_indicies = image >= 1
    image[white_indicies] = 255
    dest_image_path = os.path.join(dest_img_dir, img)
    cv2.imwrite(dest_image_path, (image).astype(np.uint8))

def generate_images(source, dest, max_rand=360, save_image_files=False):
  logger.debug("Starting to generate Images")
  bg_images = sorted(os.listdir(source['bg']))
  fg_images = sorted(os.listdir(source['fg']))
  black_image = Image.open(source['black'])
  black_image = ImageOps.fit(black_image,(448,448), method=Image.ANTIALIAS)

  bg_fg_zip = zipfile.ZipFile(os.path.join(dest,'bg_fg.zip'), mode='a', compression=zipfile.ZIP_STORED)
  bg_fg_mask_zip = zipfile.ZipFile(os.path.join(dest,'bg_fg_mask.zip'), mode='a', compression=zipfile.ZIP_STORED)

  for bg_image_name in bg_images:
    logger.debug("Background Image : {}".format(bg_image_name))
    bg_image = Image.open(os.path.join(source['bg'], bg_image_name))
    for fg_image_name in fg_images:
      logger.debug("Foreground Image : {}".format(fg_image_name))
      # https://stackoverflow.com/questions/7911451/pil-convert-png-or-gif-with-transparency-to-jpg-without
      fg_image = Image.open(os.path.join(source['fg'], fg_image_name)).convert('RGBA')
      fg_mask_image = Image.open(os.path.join(source['fg-mask'], fg_image_name)).convert('RGBA')

      fg_image_flip = ImageOps.mirror(fg_image)
      fg_mask_image_flip = ImageOps.mirror(fg_mask_image)

      partial_dest_dir = os.path.join(
          "bg-{}".format(bg_image_name.split('.')[0]),
          "fg-{}".format(fg_image_name.split('.')[0])
      )
      if save_image_files:
        dest_image_path = os.path.join(dest,"bg-fg", partial_dest_dir)
        dest_mask_path = os.path.join(dest,"bg-fg-mask", partial_dest_dir)
        os.makedirs(dest_image_path, exist_ok=True)
        os.makedirs(dest_mask_path, exist_ok=True)
      else:
        dest_temp = os.path.join(dest,"temp")
        os.makedirs(dest_temp, exist_ok=True)

      for placement in range(1,21):
        x1,y1 = random.randint(1,max_rand), random.randint(1,max_rand)
        bg_fg = copy.deepcopy(bg_image)
        mask = copy.deepcopy(black_image)
        bg_fg.paste(fg_image, (x1,y1), fg_image)
        mask.paste(fg_mask_image, (x1,y1), fg_mask_image)

        x2,y2 = random.randint(1,max_rand), random.randint(1,max_rand)
        bg_fg_flip = copy.deepcopy(bg_image)
        mask_flip = copy.deepcopy(black_image)
        bg_fg_flip.paste(fg_image_flip, (x2,y2), fg_image_flip)
        mask_flip.paste(fg_mask_image_flip, (x2,y2), fg_mask_image_flip)

        if save_image_files:
          bg_fg.save(os.path.join(dest_image_path,"{}-{}-{}.jpg".format(placement,x1,y1)))
          mask.save(os.path.join(dest_mask_path,"{}-{}-{}.jpg".format(placement,x1,y1)))

          bg_fg_flip.save(os.path.join(dest_image_path,"{}-{}-{}-flip.jpg".format(placement,x2,y2)))
          mask_flip.save(os.path.join(dest_mask_path, "{}-{}-{}-flip.jpg".format(placement,x2,y2)))

          bg_fg_zip.write(
            os.path.join(dest_image_path,"{}-{}-{}.jpg".format(placement,x1,y1)),
            arcname= os.path.join("bg-fg",partial_dest_dir,"{}-{}-{}.jpg".format(placement,x1,y1))
          )
          bg_fg_zip.write(
            os.path.join(dest_image_path,"{}-{}-{}-flip.jpg".format(placement,x2,y2)),
            arcname= os.path.join("bg-fg",partial_dest_dir,"{}-{}-{}-flip.jpg".format(placement,x2,y2))
          )

          bg_fg_mask_zip.write(
            os.path.join(dest_mask_path,"{}-{}-{}.jpg".format(placement,x1,y1)),
            arcname= os.path.join("bg-fg-mask",partial_dest_dir,"{}-{}-{}.jpg".format(placement,x1,y1))
          )
          bg_fg_mask_zip.write(
            os.path.join(dest_mask_path,"{}-{}-{}-flip.jpg".format(placement,x2,y2)),
            arcname= os.path.join("bg-fg-mask",partial_dest_dir,"{}-{}-{}-flip.jpg".format(placement,x2,y2))
          )
        else:
          bg_fg.save(os.path.join(dest_temp,"bf.jpg"))
          mask.save(os.path.join(dest_temp,"bf-mask.jpg"))

          bg_fg_flip.save(os.path.join(dest_temp,"bf-flip.jpg"))
          mask_flip.save(os.path.join(dest_temp,"bf-mask-flip.jpg"))

          bg_fg_zip.write(
            os.path.join(os.path.join(dest_temp,"bf.jpg")),
            arcname= os.path.join("bg-fg",partial_dest_dir,"{}-{}-{}.jpg".format(placement,x1,y1))
          )
          bg_fg_zip.write(
            os.path.join(os.path.join(dest_temp,"bf-flip.jpg")),
            arcname= os.path.join("bg-fg",partial_dest_dir,"{}-{}-{}-flip.jpg".format(placement,x2,y2))
          )

          bg_fg_mask_zip.write(
            os.path.join(os.path.join(dest_temp,"bf-mask.jpg")),
            arcname= os.path.join("bg-fg-mask",partial_dest_dir,"{}-{}-{}.jpg".format(placement,x1,y1))
          )
          bg_fg_mask_zip.write(
            os.path.join(os.path.join(dest_temp,"bf-mask-flip.jpg")),
            arcname= os.path.join("bg-fg-mask",partial_dest_dir,"{}-{}-{}-flip.jpg".format(placement,x2,y2))
          )

  bg_fg_zip.close()
  bg_fg_mask_zip.close()


In [0]:
cur_dir = os.curdir
generate_images({
  'bg': os.path.join(data_root,'bg/altered/'),
  'fg': os.path.join(data_root,'fg/altered/'),
  'fg-mask':os.path.join(data_root,'fg/mask/'),
  'black': os.path.join(data_root,'black.jpg')
  }
  , dest= os.path.abspath(os.path.join(os.curdir,'images/generated'))
)

In [0]:
!ls images/

In [0]:
!cp images/generated/bg_fg.zip '/content/gdrive/My Drive/Colab Notebooks/EVA4/s14-15/data/generated'

In [0]:
!cp images/generated/bg_fg_mask.zip '/content/gdrive/My Drive/Colab Notebooks/EVA4/s14-15/data/generated'